# Conditional sampling of $v$ in time

Here, we try to carefully implement Section 8.7.1 of the notes. Once this seems to be working, we'll move it off to a module.

In [ ]:
import numpy as np
np.random.seed(12345)

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

plt.rcParams.update({
    "text.usetex": True,
    'text.latex.preamble': '\\usepackage{amsmath}\\usepackage{amssymb}'
})

We'll start by definining some parameters:

In [ ]:
N = 1000  # total number of samples
dt = 0.001  # sample spacing (in s)
taxis = np.arange(0, N * dt, dt)
freqs = np.fft.fftfreq(N, dt)
Omega = len(freqs)

Now we want to write a function that calculates $\boldsymbol{\phi}_t$ from the notes:

In [ ]:
def phi(t, freqs):
    """
    Calculate phi_t as in notes. 
    Assumes \omega = 0 is at index 0!
    """
    phase = np.exp(1j * 2 * np.pi * freqs * t)
    phase[0] -= 1
    phase /= np.sqrt(freqs.shape[0])
    return phase

A simple test: We should have $\mathrm{iFFT}(x)(t) = \frac{\sqrt{\Omega - 1}}{\Omega}\boldsymbol{\phi}_t \cdot \mathbf{x}_t$ when $\mathbf{x}$ has no zero-frequency content.

In [ ]:
x = np.random.randn(Omega)
x[0] = 0
ifftx = np.fft.ifft(x)

phix = (np.sqrt(Omega - 1)/Omega) * np.array([np.dot(phi(t, freqs), x) for t in taxis])

In [ ]:
ifftx[:10], phix[:10]

In [ ]:
fig, ax = plt.subplots(1, 2)
ax[0].plot(taxis, np.real(ifftx - phix))
ax[0].set_title(r"$\mathcal{R}[\mathrm{iFFT}(x)(t) - \frac{\sqrt{\Omega - 1}}{\Omega}\boldsymbol{\phi}_t \cdot \mathbf{x}_t]$")
ax[0].set_xlabel("Time (s)")
ax[1].plot(taxis, np.imag(ifftx - phix))
ax[1].set_title(r"$\mathcal{I}[\mathrm{iFFT}(x)(t) - \frac{\sqrt{\Omega - 1}}{\Omega}\boldsymbol{\phi}_t \cdot \mathbf{x}_t]$")
ax[1].set_xlabel("Time (s)");